# precompile

> A tool to pre-compile Numba functions for speeding up DABEST bootstrapping

- order: 10

In [ ]:
#| default_exp _stats_tools/precompile

In [ ]:
#| hide
from __future__ import annotations

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev
nbdev.nbdev_export()

In [ ]:
#| export
import numpy as np
from tqdm import tqdm
from dabest._stats_tools import effsize
from dabest._stats_tools import confint_2group_diff

In [ ]:
#| export

_NUMBA_COMPILED = False

def precompile_all():
    """Pre-compile all numba functions with dummy data"""
    global _NUMBA_COMPILED
    
    if _NUMBA_COMPILED:
        return
        
    print("Pre-compiling numba functions for DABEST...")
    
    # Create dummy data
    dummy_control = np.array([1.0, 2.0, 3.0])
    dummy_test = np.array([4.0, 5.0, 6.0])
    
    funcs = [
        # effsize.py functions
        (effsize.cohens_d, (dummy_control, dummy_test)),
        (effsize._mann_whitney_u, (dummy_control, dummy_test)),
        (effsize._cliffs_delta_core, (dummy_control, dummy_test)),
        (effsize._compute_standardizers, (dummy_control, dummy_test)),
        (effsize.weighted_delta, (np.array([1.0, 2.0]), np.array([0.1, 0.2]))),
        
        # confint_2group_diff.py functions
        (confint_2group_diff.create_jackknife_indexes, (dummy_control,)),
        (confint_2group_diff.create_repeated_indexes, (dummy_control,)),
        (confint_2group_diff.bootstrap_indices, (True, 3, 3, 10, 12345)),
        (confint_2group_diff.delta2_bootstrap_loop, 
            (dummy_control, dummy_test, dummy_control, dummy_test, 10, 1.0, 12345, False)),
        (confint_2group_diff._compute_quantile, (0.5, 0.1, 0.1)),
        (confint_2group_diff.calculate_group_var, (1.0, 3, 1.0, 3))
    ]
    
    for func, args in tqdm(funcs, desc="Compiling numba functions"):
        func(*args)
    
    _NUMBA_COMPILED = True
    
    print("Numba compilation complete!")